<a href="https://colab.research.google.com/github/weladmani/Stroke-Prediction-on-Kaggle-dataset/blob/main/Maman22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries for Basic Data Processing
import numpy as np
import pandas as pd

#Libraries for File I/O Operations
import os

import requests
import itertools

#Libraries for Exception Handling
import warnings
warnings.filterwarnings('ignore')

#Libraries for data pre-processing, visualization and analysis
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.cluster import KMeans

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from mlxtend.frequent_patterns import apriori, association_rules

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tqdm.notebook import tqdm

In [ ]:
#Read data and get its structure, features and description
import kagglehub

path = kagglehub.dataset_download("fedesoriano/stroke-prediction-dataset")
data = pd.read_csv(os.path.join(path, 'healthcare-dataset-stroke-data.csv'))

print(f"\nGlimpse Of The Dataset :")
data.head(10)

In [ ]:
# === STEP 1: Load & Clean Data ===
data = data[data['gender'] != 'Other']
data.drop(columns=['id'], inplace=True)
data['bmi'].fillna(data['bmi'].median(), inplace=True)

# === STEP 2: Bin Continuous Variables ===

# Age groups
data['age_group'] = pd.cut(
    data['age'],
    bins=[0, 25, 50, 75, 120],
    labels=['0-25', '26-50', '51-75', '75+']
)

# BMI groups
data['bmi_group'] = pd.cut(
    data['bmi'],
    bins=[0, 18.5, 25, 30, 100],
    labels=['Underweight', 'Healthy', 'Overweight', 'Obese']
)

# Glucose level groups
data['glucose_group'] = pd.cut(
    data['avg_glucose_level'],
    bins=[0, 140, 200, data['avg_glucose_level'].max()],
    labels=['Normal', 'Prediabetes', 'Diabetes']
)

# === STEP 3: Drop Original Continuous Columns ===
data.drop(columns=['age', 'bmi', 'avg_glucose_level'], inplace=True)

In [ ]:
# === STEP 4: Convert Binary Columns to Boolean ===
data['hypertension'] = data['hypertension'].astype(bool)
data['heart_disease'] = data['heart_disease'].astype(bool)
data['ever_married'] = data['ever_married'].map({'Yes': True, 'No': False})
data['stroke'] = data['stroke'].astype(bool)

In [ ]:
# === STEP 5: One-Hot Encode Categorical Features ===
categorical_cols = ['gender', 'work_type', 'Residence_type', 'smoking_status', 'age_group', 'bmi_group', 'glucose_group']
data_encoded = pd.get_dummies(data, columns=categorical_cols)

In [ ]:
# === STEP 6: Prepare for SMOTENC ===
X = data_encoded.drop('stroke', axis=1)
y = data_encoded['stroke']

# Identify which columns are categorical (one-hot encoded start index)
categorical_features = [i for i, col in enumerate(X.columns) if any(prefix in col for prefix in [
    'gender_', 'work_type_', 'Residence_type_', 'smoking_status_', 'age_group_', 'bmi_group_', 'glucose_group_'
])]

smote = SMOTENC(categorical_features=categorical_features, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Combine into balanced DataFrame
data_balanced = pd.DataFrame(X_resampled, columns=X.columns)
data_balanced['stroke'] = y_resampled.astype(bool)

In [ ]:
# === STEP 7: Convert All Columns to Boolean for Apriori ===
data_apriori = data_balanced.astype(bool)

# === STEP 8: Run Apriori Algorithm ===
freq_items = apriori(data_apriori, min_support=0.4, use_colnames=True)
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)

# === STEP 9: Filter or Sort Rules ===
rules_filteredall = rules.sort_values(by='confidence', ascending=False)
rules_filtered = rules[(rules['lift'] > 1.1)].sort_values(by='confidence', ascending=False)

In [ ]:
# === STEP 10: Display All Rules ===
print("All Association Rules (lift > 1.1):\n")
for _, row in rules_filteredall.iterrows():
    antecedents = ', '.join([str(i) for i in row['antecedents']])
    consequents = ', '.join([str(i) for i in row['consequents']])
    print(f"If {{{antecedents}}} → {{{consequents}}} | "
          f"support: {row['support']:.2f}, "
          f"confidence: {row['confidence']:.2f}, "
          f"lift: {row['lift']:.2f}")

In [ ]:
# === STEP 11: Display Top Rules ===
print("Top Association Rules (lift > 1.1):\n")
for _, row in rules_filtered.head(10).iterrows():
    antecedents = ', '.join([str(i) for i in row['antecedents']])
    consequents = ', '.join([str(i) for i in row['consequents']])
    print(f"If {{{antecedents}}} → {{{consequents}}} | "
          f"support: {row['support']:.2f}, "
          f"confidence: {row['confidence']:.2f}, "
          f"lift: {row['lift']:.2f}")

# results for question 1

The results show clear statistical associations between variables such as age, marital status, and stroke. However, it is important to note that these relationships are likely influenced by a third variable — in this case, age. Older individuals are more likely both to be married and to have experienced a stroke.

Question 2 - Clustering Analysis

A. To evaluate the quality of the clustering I performed on the Stroke Prediction Dataset, I will use two measures: Silhouette Score and Normalized Mutual Information (NMI).

1. **Silhouette Score**: The Silhouette Score is an internal evaluation metric, meaning it only looks at the structure of the clusters, without using the actual labels.
It measures how similar each data point is to its own cluster compared to other clusters.
The score ranges from -1 to 1, where values closer to 1 indicate well-defined and separated clusters.
I used this measure to check whether the clustering was meaningful based on the features in the dataset.

2. **Normalized Mutual Information**: NMI is an external metric that compares the cluster labels to the actual labels in the dataset (in this case, whether or not the person had a stroke).
It measures how much information is shared between the clusters and the real labels.
The score also ranges from 0 to 1, where 1 means a perfect match between the clusters and the actual stroke labels.
This helps us understand if the clusters we found are related to the stroke outcome in a meaningful way.

B. **Clustering Method: K-Means**:
K-Means is a popular unsupervised machine learning algorithm that groups the data into K clusters based on similarity.
It works by:
1. Selecting K cluster centers (called centroids).
2. Assigning each data point to the closest centroid.
3. Recalculating the centroids based on the assigned points.
4. Repeating the process until the assignments no longer change significantly.

I chose K-means becuase it is easy to understand, quick to implement, works well with large datasets and is compatibile with our dataset since most features are numeric (after encoding)

In [ ]:
data_2 = pd.read_csv(os.path.join(path, 'healthcare-dataset-stroke-data.csv'))

# Drop Unnecessary Columns
data_2.drop(['id'], axis=1, inplace=True)
# Handle Missing Values
data_2['bmi'].fillna(data_2['bmi'].median(), inplace=True)
# Encode Categorical Variables
data_encoded = pd.get_dummies(data_2, drop_first=True)

#Feature Scaling
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_encoded)


# Apply K-Means Clustering
I will use the Elbow method to Find the Optimal K

In [ ]:
import matplotlib.pyplot as plt

inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal K')
plt.grid(True)
plt.show()


Optimal K would be K=3

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Add the clusters back to the DataFrame:
data_2['Cluster'] = clusters


In [ ]:
from sklearn.metrics import silhouette_score

# Evaluate Clustering Quality

sil_score = silhouette_score(data_scaled, clusters)
print(f"Silhouette Score: {sil_score:.2f}")

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

# Normalized Mutual Information

nmi_score = normalized_mutual_info_score(data_2['stroke'], data_2['Cluster'])
print(f"NMI Score: {nmi_score:.2f}")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce data to 2 principal components
pca = PCA(n_components=2)
pca_components = pca.fit_transform(data_scaled)

# Plot clusters in 2D
plt.figure(figsize=(8, 6))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=data_2['Cluster'], cmap='viridis', alpha=0.6)
plt.title('K-Means Clusters (K=3) Visualized with PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster')
plt.grid(True)
plt.show()


In [ ]:
# Average values per cluster
summary = data_2.groupby('Cluster').mean(numeric_only=True).round(2)

# Stroke proportion per cluster
stroke_distribution = data_2.groupby('Cluster')['stroke'].value_counts(normalize=True).unstack().fillna(0)
stroke_distribution.columns = ['No Stroke', 'Stroke']


print("Feature Averages by Cluster:\n", summary)
print("\nStroke Proportion per Cluster:\n", stroke_distribution)

**Final Assumptions**

After applying K-Means clustering with K=3, we notice three distinct groups:

Cluster 1: Young individuals (avg. age 7), with no hypertension or heart disease, low glucose and BMI. Very low stroke risk (0.3%). This is a low-risk group.

Clusters 0 & 2: Middle-aged individuals (avg. age 47-49), with moderate levels of hypertension, heart disease, and higher BMI and glucose. Both show similar stroke rates (~ 5-6%), and represent the moderate- to high-risk groups.

# Artificial Neural Network
A.
Since this is a binary classification problem (stroke = 1 or 0) I will define a feedforward ANN with:


    Layer                  | Neurons | Activation | Purpose
    -----------------------|---------|------------|----------------
    Input + Hidden layer 1 | 32      | ReLU       | Detects patterns in features like age, BMI, etc.
    Hidden Layer 2         | 16      | ReLU       | Learns deeper/more abstract
    interactions.
    Hidden Layer 3         | 8       | ReLU       | Learns deeper/more abstract
    interactions.
    Output Layer	       | 1       | Sigmoid    | Outputs probability of stroke (0-1)





*   All layers are fully connected - each neuron in one layer connects to every neuron in the next.
*   The number of input neurons = number of features after preprocessing

**Data Flow**

1.   Input: A row from the dataset is passed to the first layer.
2.   Each neuron computes:
      z=∑(w
i
​
 ⋅x
i
​
 )+b
then applies the activation function to get the output.
3. The result flows to the next layer and the process repeats.
4. Output neuron returns a probability between 0 and 1.

**Activation Functions**

* ReLU (Rectified Linear Unit): f(x) = max(0,x)
Helps prevent vanishing gradients and speeds up training.
* Sigmoid: f(x) = 1 / (1 + e^-x)
Used at the output layer to produce a value betweeb 0 and 1 -> perfect for binary classification.

B.
* Loss Function: Loss = -[y * log(y^) + (1 - y) * log(1 - y^)]

* Batch Size: 32, the number of training samples  processed before updating the model's weights.

* Learning Rate: will use Adam optimizer, which automaticallty adjusts the learning arete for each parameter.

The optimization process uses binary cross-entropy as the loss function to measure the prediction error for stroke vs. non-stroke.
The model is trained using the Adam optimizer, which adaptively adjusts the learning rate during training.
I use a batch size of 32, meaning the model updates weights after processing every 32 examples.
The default learning rate for Adam is 0.001, which provides a good balance between speed and convergence stability.



In [ ]:
# Progress bar callback using tqdm
class TQDMProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.tqdm = tqdm(total=self.epochs, desc="Training Progress")
    def on_epoch_end(self, epoch, logs=None):
        self.tqdm.update(1)
    def on_train_end(self, logs=None):
        self.tqdm.close()

# Load and preprocess dataset
data = pd.read_csv(os.path.join(path, 'healthcare-dataset-stroke-data.csv'))
data.drop(['id'], axis=1, inplace=True)
data['bmi'].fillna(data['bmi'].median(), inplace=True)
data_encoded = pd.get_dummies(data, drop_first=True)

X = data_encoded.drop('stroke', axis=1)
y = data_encoded['stroke']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define ANN model
model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=48,
    batch_size=32,
    verbose=0,
    callbacks=[TQDMProgressBar()]
)

# Evaluate
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ Recall (stroke cases):", recall_score(y_test, y_pred))
print("\n🧾 Classification Report:\n", classification_report(y_test, y_pred))
print("📉 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Plot Loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Binary Cross-Entropy Loss')
plt.title('Training and Test Loss Over Epochs (With SMOTE)')
plt.legend()
plt.grid(True)
plt.show()

We can see that the Loss is decreasing with every Epoch.

In [ ]:
# Get indices of false positives: predicted 1, actual 0
# Reset index of y_test to align with X_test_unscaled
false_positive_idx = (y_pred == 1) & (y_test.reset_index(drop=True) == 0)

# Recover original feature names from the encoded data before scaling
feature_names = X.columns

# Inverse transform to get original (unscaled) feature values
# Fit the scaler on the scaled training data used for the model
scaler.fit(X_scaled)
X_test_unscaled = pd.DataFrame(scaler.inverse_transform(X_test), columns=feature_names)

# Get corresponding false positive rows
false_positives = X_test_unscaled[false_positive_idx].copy()
false_positives['Predicted_Prob'] = y_pred_prob[false_positive_idx]

# Print all false positives
print(f"\n🔎 Total False Positives: {false_positives.shape[0]}\n")
from IPython.display import display
display(false_positives.head(10))

# Get indices of false negatives: predicted 0, actual 1
false_negative_idx = (y_pred == 0) & (y_test.reset_index(drop=True) == 1)

# Inverse transform to get original (unscaled) feature values
false_negatives = X_test_unscaled[false_negative_idx].copy()
false_negatives['Predicted_Prob'] = y_pred_prob[false_negative_idx]

# Print all false negatives
print(f"\n⚠️ Total False Negatives: {false_negatives.shape[0]}\n")
display(false_negatives.head(10))

**analyzing and assumptions**

C. The final ANN model achieved 87% accuracy with an excellent 97% recall on stroke cases. This suggests the model is highly effective at identifying patients at risk, with a low false negative rate. While false positives are present (~20%), this is often acceptable in medical screening where early detection is prioritized. The model was trained on SMOTE-balanced data, and both training and test loss show stable convergence without overfitting. Future improvements can aim to refine precision while preserving this strong sensitivity.

# Final Summary – Stroke Prediction Project

This project combined classification models, association rule mining, clustering, and neural networks to explore and predict stroke risk in a real-world medical dataset.

**Phase 1: Supervised Learning**


*   Random Forest and Logistic Regression were applied initially.
*   Logistic Regression was selected as the best model due to its strong recall and balanced performance, making it suitable for stroke detection where missing a case is critical.

**Phase 2: Association Rule Mining**
Key patterns revealed:


*   Stroke → Ever Married (Confidence: 97%, Lift: 1.20)
*   Age 51–75 → Ever Married (Confidence: 96%)
*   Ever Married → Stroke (Confidence: 60%, Lift: 1.20)

These rules suggest that age is a hidden confounder, influencing both marital status and stroke likelihood.

**Phase 3: Clustering (K-Means, K=3)**
Identified three distinct groups:


*   Low-risk: Very young, no health risk factors.
*   Moderate- to high-risk: Middle-aged, higher BMI, glucose, and comorbidities.

Reinforced age, BMI, and hypertension as key stroke predictors.


**Phase 4: ANN + SMOTE**
Final results:
* Accuracy: 87%
* Recall (stroke cases): 98%
* Precision: 80%
* Only 19 false negatives → strong detection power




# Final Conclusion

Based on all the results, the ANN with SMOTE is the strongest model tested so far, especially for high recall, making it the most suitable for stroke risk detection.